# 🚀 HVAC AI — Production-Ready YOLOv11-OBB Pipeline
**End-to-End Oriented Bounding Box (OBB) Analysis & Inference Server**

---

## 📋 Overview
This notebook deploys a high-performance API for HVAC blueprint analysis.

### 🌟 Configuration
- **Endpoint**: `/api/v1/analyze/stream` (POST)
- **Format**: YOLOv11-OBB (Rotated Bounding Boxes)
- **Optimization**: FP16 (Half-Precision) & CUDA
- **Security**: CORS enabled for Frontend access

## 🎯 Instructions
1. Set Runtime to **GPU**.
2. Run all cells from top to bottom.
3. Copy the **Ngrok Public URL** into your Frontend `.env` file.

In [ ]:
print("🔧 Environment Setup")
print("="*70)

# Mount Drive
from google.colab import drive
import os
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Install Dependencies
print("\n📦 Installing libraries...")
!pip install -q ultralytics>=8.3.0 fastapi>=0.115.0 uvicorn[standard]>=0.34.0
!pip install -q python-multipart pyngrok>=7.0.0 python-dotenv opencv-python-headless

import torch
import sys
print(f"\n✅ Python: {sys.version.split()[0]}")
print(f"✅ PyTorch: {torch.__version__}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  WARNING: Running on CPU (Slow)")

In [ ]:
print("⚙️  Pipeline Configuration")
print("="*70)

# --- USER CONFIGURATION START ---
# Path to your YOLOv11-OBB model
MODEL_PATH = "/content/drive/Shareddrives/HVAC/DECEMBER 24 OUTPUT WEIGHTS {dataset2}/hvac_obb_l_20251224_214011/weights/best.pt"

# Inference Settings
CONF_THRESHOLD = 0.50      # Confidence threshold
IOU_THRESHOLD = 0.45       # NMS IoU threshold
IMG_SIZE = 1024            # Inference image size
HALF_PRECISION = torch.cuda.is_available() # Use FP16 if GPU is available

# Server Settings
PORT = 8000
NGROK_AUTHTOKEN = "36hBoLt4A3L8yOYt96wKiCxxrwp_5wFbj1Frv6GoHARRQ6H6t" # <--- PASTE TOKEN HERE
# --- USER CONFIGURATION END ---

if not os.path.exists(MODEL_PATH):
    print(f"❌ ERROR: Model not found at {MODEL_PATH}")
    print("   Please update MODEL_PATH to point to your .pt file.")
else:
    print(f"✅ Model Path: {MODEL_PATH}")
    print(f"✅ Config: Conf={CONF_THRESHOLD}, IoU={IOU_THRESHOLD}, FP16={HALF_PRECISION}")

In [ ]:
print("📝 Generating Production Server Code (app.py)...")
print("="*70)

# Updates:
# 1. STRICT OBB MODE: Extracts 'xywhr' (Center X, Center Y, Width, Height, Rotation).
# 2. Flattens the response: Puts x, y, w, h, r at the top level of the detection object.
# 3. Removes generic 'bbox' or 'polygon' keys to prevent frontend confusion.

server_code = f"""
import uvicorn
from fastapi import FastAPI, UploadFile, File, HTTPException, Request
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import numpy as np
import cv2
import torch
from ultralytics import YOLO
import logging
import sys

# --- CONFIGURATION ---
MODEL_PATH = r'{MODEL_PATH}'
CONF_THRES = {CONF_THRESHOLD}
IOU_THRES = {IOU_THRESHOLD}
IMG_SIZE = {IMG_SIZE}
HALF = {HALF_PRECISION}

# --- LOGGING ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger("HVAC-Service")

app = FastAPI(title="HVAC YOLOv11-OBB Inference API")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

model = None

@app.on_event("startup")
async def load_model():
    global model
    logger.info(f"Loading OBB model from {{MODEL_PATH}}...")
    try:
        model = YOLO(MODEL_PATH)
        if torch.cuda.is_available():
            model.to('cuda')
            logger.info("Model loaded on GPU")
        else:
            logger.info("Model loaded on CPU")
        # Warmup
        model.predict(np.zeros((640,640,3), dtype=np.uint8), verbose=False, half=HALF)
        logger.info("Model Warmup Complete")
    except Exception as e:
        logger.error(f"Failed to load model: {{e}}")
        raise RuntimeError("Model loading failed")

@app.get("/")
def root():
    return {{"message": "HVAC OBB Inference Server Online", "docs": "/docs"}}

@app.get("/health")
def health_check():
    if model is None:
        raise HTTPException(status_code=503, detail="Model initializing")
    return {{"status": "healthy", "device": str(model.device), "task": "obb"}}

@app.get("/api/v1/analyze/stream")
def analyze_help():
    return JSONResponse(status_code=200, content={{"message": "Ready for OBB Inference"}})

@app.post("/api/v1/analyze/stream")
async def analyze_image(
    request: Request,
    file: UploadFile = File(None),
    image: UploadFile = File(None)
):
    if not model:
        raise HTTPException(status_code=503, detail="Model not loaded")

    target_file = file or image
    if not target_file:
        raise HTTPException(status_code=422, detail="No file uploaded. Use key 'file' or 'image'.")

    try:
        logger.info(f"📸 Processing OBB Request: {{target_file.filename}}")
        contents = await target_file.read()
        nparr = np.frombuffer(contents, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        if img is None:
            raise HTTPException(status_code=400, detail="Invalid image format")

        # Inference
        results = model.predict(
            img,
            conf=CONF_THRES,
            iou=IOU_THRES,
            imgsz=IMG_SIZE,
            half=HALF
        )

        result = results[0]
        detections = []

        # --- STRICT OBB FORMATTING ---
        if hasattr(result, 'obb') and result.obb is not None:
            for box in result.obb:
                # Extract OBB Data: [x_center, y_center, width, height, rotation]
                xywhr = box.xywhr[0].cpu().numpy().tolist()
                cls_id = int(box.cls[0].item())
                conf = float(box.conf[0].item())
                label = result.names[cls_id]

                detections.append({{
                    "label": label,          # Common Frontend Key
                    "class": label,          # Common Frontend Key
                    "score": conf,           # Common Frontend Key
                    "confidence": conf,
                    "type": "OBB",

                    # Flattened OBB Data (Easiest for Frontend to read)
                    "x": xywhr[0],           # Center X
                    "y": xywhr[1],           # Center Y
                    "w": xywhr[2],           # Width
                    "h": xywhr[3],           # Height
                    "rotation": xywhr[4],    # Rotation (Radians)
                    "r": xywhr[4],           # Alias for rotation

                    # Explicit Object for strict types
                    "obb": {{
                        "x_center": xywhr[0],
                        "y_center": xywhr[1],
                        "width": xywhr[2],
                        "height": xywhr[3],
                        "rotation": xywhr[4]
                    }}
                }})
        else:
            logger.warning("⚠️ No OBB detections found (or model is not OBB)")

        logger.info(f"✅ Success: {{len(detections)}} OBB components detected")

        return JSONResponse(content={{
            "success": True,
            "count": len(detections),
            "detections": detections
        }})

    except Exception as e:
        logger.error(f"Inference error: {{e}}")
        raise HTTPException(status_code=500, detail=str(e))
"""

with open("app.py", "w") as f:
    f.write(server_code)

print("✅ Generated app.py successfully")

In [ ]:
import subprocess
import time
import requests
import sys
from pyngrok import ngrok

print("🚀 Launching & Validating Server")
print("="*70)

# 1. Cleanup
ngrok.kill()

# 2. Start Uvicorn
print("⏳ Starting Uvicorn process...")
process = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "app:app", "--host", "0.0.0.0", "--port", str(PORT)],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    encoding='utf-8',
    bufsize=1
)

# 3. Health Check
print("🏥 Checking server health (timeout: 60s)...\n")
server_ready = False
health_url = f"http://localhost:{PORT}/health"

start_time = time.time()
while time.time() - start_time < 60:
    try:
        response = requests.get(health_url, timeout=1)
        if response.status_code == 200:
            data = response.json()
            print(f"\n✅ Server is HEALTHY!")
            print(f"   Status: {data['status']}")
            print(f"   Device: {data['device']}")
            server_ready = True
            break
    except requests.exceptions.ConnectionError:
        print(".", end="", flush=True)
        time.sleep(2)
    except Exception as e:
        print(f"\n⚠️ Unexpected error: {e}")

if not server_ready:
    print("\n❌ Server failed to start.")
    print(process.stdout.read())
    process.terminate()
    raise RuntimeError("Server startup failed")

# 4. Ngrok Tunnel
print("\n🌐 Initializing Public Tunnel...")
if NGROK_AUTHTOKEN and NGROK_AUTHTOKEN != "YOUR_NGROK_TOKEN_HERE":
    try:
        ngrok.set_auth_token(NGROK_AUTHTOKEN)
        tunnel = ngrok.connect(PORT)
        public_url = tunnel.public_url
        print(f"\n🎉 API IS LIVE at: {public_url}")
        print(f"   📄 Docs: {public_url}/docs")
        print(f"   🔗 Endpoint: {public_url}/api/v1/analyze/stream")
    except Exception as e:
        print(f"⚠️ Ngrok Error: {e}")
else:
    print("⚠️ No Ngrok token. Server local only.")

print("\n📜 Streaming Logs (Press STOP to exit)...")
print("-" * 70)

# 5. Log Loop
try:
    while True:
        line = process.stdout.readline()
        if line:
            print(line.strip())
        if process.poll() is not None:
            print("❌ Server process terminated.")
            break
except KeyboardInterrupt:
    print("\n🛑 Server stopped by user.")
    process.terminate()
    ngrok.kill()